### Install & Import Packages

In [3]:
%%capture
!pip install "flair" -q

In [4]:
from google.colab import drive
drive.mount('/content/drive')
import flair
from flair.data import Sentence
from flair.datasets import ColumnCorpus
from flair.embeddings import (
    WordEmbeddings, TransformerWordEmbeddings, StackedEmbeddings
)
from flair.models import SequenceTagger
from flair.trainers import ModelTrainer
flair.__version__

'0.12.2'

### Import Data

In [5]:
DATA_PATH = "/content/drive/Shareddrives/CIS522-Project/data"
MODEL_PATH = "/content/drive/Shareddrives/CIS522-Project/models"

In [6]:
# Import data into flair using ColumnCorpus
corpus = ColumnCorpus(DATA_PATH, {0: "text", 1: "ner"}, train_file="flair_ner_train_augmented.txt", test_file="flair_ner_test.txt")
# Needed for model initialization
tag_dictionary = corpus.make_label_dictionary(label_type="ner", add_unk=False)
print(tag_dictionary.get_items())

2023-04-18 22:38:59,855 Reading data from /content/drive/Shareddrives/CIS522-Project/data
2023-04-18 22:38:59,862 Train: /content/drive/Shareddrives/CIS522-Project/data/flair_ner_train_augmented.txt
2023-04-18 22:38:59,863 Dev: None
2023-04-18 22:38:59,866 Test: /content/drive/Shareddrives/CIS522-Project/data/flair_ner_test.txt
2023-04-18 22:39:27,841 Computing label dictionary. Progress:


2776it [00:00, 35853.94it/s]

2023-04-18 22:39:27,957 Dictionary created for label 'ner' with 9 values: Drug (seen 5609 times), ADE (seen 4577 times), Reason (seen 1269 times), Strength (seen 972 times), Route (seen 824 times), Frequency (seen 723 times), Form (seen 620 times), Dosage (seen 598 times), Duration (seen 121 times)
['Drug', 'ADE', 'Reason', 'Strength', 'Route', 'Frequency', 'Form', 'Dosage', 'Duration']


### Initialize Weight Dictionary
This is the weight dictionary used by the loss function. The weights for the ADE and Reason entities are set to 5, and for the remaining entities are set to 1.

In [7]:
weight_dict = {
    'Drug': 1,
    'Strength': 1,
    'Form': 1,
    'Frequency': 1,
    'Route': 1,
    'Dosage': 1,
    'Reason': 5,
    'Duration': 1,
    'ADE': 5
}

In [8]:
tf_tagger = SequenceTagger.load(f"{MODEL_PATH}/taggers/clinicalbert-crf/final-model.pt")
tf_tagger.weight_dict = weight_dict

2023-04-18 22:39:45,747 SequenceTagger predicts: Dictionary with 39 tags: O, S-Drug, B-Drug, E-Drug, I-Drug, S-Strength, B-Strength, E-Strength, I-Strength, S-Form, B-Form, E-Form, I-Form, S-Frequency, B-Frequency, E-Frequency, I-Frequency, S-Route, B-Route, E-Route, I-Route, S-Dosage, B-Dosage, E-Dosage, I-Dosage, S-Reason, B-Reason, E-Reason, I-Reason, S-Duration, B-Duration, E-Duration, I-Duration, S-ADE, B-ADE, E-ADE, I-ADE, <START>, <STOP>


### Fine-Tune & Evaluate Model

In [9]:
# Initialize trainer
trainer = ModelTrainer(tf_tagger, corpus)

# Train on corpus
trainer.train(
    base_path=f"{MODEL_PATH}/taggers/clinicalbert-crf-augmented-weights",
    train_with_dev=False,
    max_epochs=10,
    learning_rate=0.005,
    mini_batch_size=16,
    embeddings_storage_mode='none'
)

/usr/local/lib/python3.9/dist-packages/flair/trainers/trainer.py:69: UserWarning: There should be no best model saved at epoch 1 except there is a model from previous trainings in your training folder. All previous best models will be deleted.
  warnings.warn(


2023-04-18 22:39:47,242 ----------------------------------------------------------------------------------------------------
2023-04-18 22:39:47,246 Model: "SequenceTagger(
  (embeddings): StackedEmbeddings(
    (list_embedding_0): TransformerWordEmbeddings(
      (model): BertModel(
        (embeddings): BertEmbeddings(
          (word_embeddings): Embedding(28997, 768)
          (position_embeddings): Embedding(512, 768)
          (token_type_embeddings): Embedding(2, 768)
          (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (dropout): Dropout(p=0.1, inplace=False)
        )
        (encoder): BertEncoder(
          (layer): ModuleList(
            (0-11): 12 x BertLayer(
              (attention): BertAttention(
                (self): BertSelfAttention(
                  (query): Linear(in_features=768, out_features=768, bias=True)
                  (key): Linear(in_features=768, out_features=768, bias=True)
                  (value): Linear(in_fea

100%|██████████| 20/20 [00:10<00:00,  2.00it/s]


2023-04-18 22:43:59,304 Evaluating as a multi-label problem: False
2023-04-18 22:43:59,353 DEV : loss 0.1806982308626175 - f1-score (micro avg)  0.8479
2023-04-18 22:43:59,371 BAD EPOCHS (no improvement): 0
2023-04-18 22:43:59,379 saving best model
2023-04-18 22:44:03,933 ----------------------------------------------------------------------------------------------------
2023-04-18 22:44:27,991 epoch 2 - iter 17/174 - loss 0.23779200 - time (sec): 24.06 - samples/sec: 441.09 - lr: 0.005000
2023-04-18 22:44:51,723 epoch 2 - iter 34/174 - loss 0.23879142 - time (sec): 47.79 - samples/sec: 442.48 - lr: 0.005000
2023-04-18 22:45:13,410 epoch 2 - iter 51/174 - loss 0.23540799 - time (sec): 69.47 - samples/sec: 453.23 - lr: 0.005000
2023-04-18 22:45:43,675 epoch 2 - iter 68/174 - loss 0.22577977 - time (sec): 99.74 - samples/sec: 438.70 - lr: 0.005000
2023-04-18 22:46:07,189 epoch 2 - iter 85/174 - loss 0.22348648 - time (sec): 123.25 - samples/sec: 441.52 - lr: 0.005000
2023-04-18 22:46:33,

100%|██████████| 20/20 [00:10<00:00,  1.91it/s]

...

2023-04-18 23:23:15,904 Evaluating as a multi-label problem: False
2023-04-18 23:23:15,931 DEV : loss 0.07097325474023819 - f1-score (micro avg)  0.9479
2023-04-18 23:23:15,943 BAD EPOCHS (no improvement): 0
2023-04-18 23:23:15,950 saving best model


2023-04-18 23:23:19,286 ----------------------------------------------------------------------------------------------------
2023-04-18 23:23:24,500 SequenceTagger predicts: Dictionary with 39 tags: O, S-Drug, B-Drug, E-Drug, I-Drug, S-Strength, B-Strength, E-Strength, I-Strength, S-Form, B-Form, E-Form, I-Form, S-Frequency, B-Frequency, E-Frequency, I-Frequency, S-Route, B-Route, E-Route, I-Route, S-Dosage, B-Dosage, E-Dosage, I-Dosage, S-Reason, B-Reason, E-Reason, I-Reason, S-Duration, B-Duration, E-Duration, I-Duration, S-ADE, B-ADE, E-ADE, I-ADE, <START>, <STOP>


100%|██████████| 1588/1588 [14:25<00:00,  1.83it/s]


2023-04-18 23:37:51,163 Evaluating as a multi-label problem: False
2023-04-18 23:37:53,272 0.8847	0.9185	0.9013	0.8278
2023-04-18 23:37:53,278 
Results:
- F-score (micro) 0.9013
- F-score (macro) 0.8149
- Accuracy 0.8278

By class:
              precision    recall  f1-score   support

        Drug     0.8697    0.9400    0.9035     61167
    Strength     0.9315    0.9573    0.9443     42957
        Form     0.9221    0.9199    0.9210     41417
   Frequency     0.8543    0.8591    0.8567     36495
       Route     0.9502    0.9449    0.9475     30583
      Dosage     0.9261    0.9432    0.9345     23506
      Reason     0.7297    0.7431    0.7363      9533
         ADE     0.2168    0.6759    0.3283      1299
    Duration     0.7432    0.7810    0.7616      1982

   micro avg     0.8847    0.9185    0.9013    248939
   macro avg     0.7937    0.8627    0.8149    248939
weighted avg     0.8923    0.9185    0.9044    248939

2023-04-18 23:37:53,279 ---------------------------------------

{'test_score': 0.9012660522975776,
 'dev_score_history': [0.8478642480983032,
  0.8683901292596946,
  0.8923346986541837,
  0.9022248243559718,
  0.9158604514805043,
  0.9214536928487691,
  0.926629640456007,
  0.9353321575543798,
  0.9389671361502349,
  0.9478952016485134],
 'train_loss_history': [0.26979004768039777,
  0.220478675166595,
  0.19417269020893674,
  0.1737861580474725,
  0.15649715688922705,
  0.14682087960186904,
  0.1336496293331674,
  0.12747490114608317,
  0.1220010997321469,
  0.11548307869139476],
 'dev_loss_history': [0.1806982308626175,
  0.14165449142456055,
  0.12228060513734818,
  0.11125482618808746,
  0.10138115286827087,
  0.09453363716602325,
  0.09184014052152634,
  0.08141843974590302,
  0.07748273760080338,
  0.07097325474023819]}

In [ ]:
# Create example sentence
sentence = Sentence("Patients on 40 mg of Topelfate and Topoxy twice a day for stomachache generally suffer from headache")

# Token level predictions
tf_tagger.predict(sentence, force_token_predictions=True)
print(sentence.to_tagged_string())

# Predict tags and print
tf_tagger.predict(sentence)
print(sentence.to_tagged_string())

Sentence[17]: "Patients on 40 mg of Topelfate and Topoxy twice a day for stomachache generally suffer from headache" → ["40"/B-Strength, "mg"/E-Strength, "Topelfate"/S-Drug, "Topoxy"/S-Drug, "twice"/B-Frequency, "a"/I-Frequency, "day"/E-Frequency, "stomachache"/S-Reason, "headache"/S-ADE]
Sentence[17]: "Patients on 40 mg of Topelfate and Topoxy twice a day for stomachache generally suffer from headache" → ["40 mg"/Strength, "Topelfate"/Drug, "Topoxy"/Drug, "twice a day"/Frequency, "stomachache"/Reason, "headache"/ADE]
